#### Importación de paquetes

In [1]:
import cv2
import dlib


#### Aplicación de filtro divertido con detector facial basado en Convolutional Neural Networks (CNNs)

In [2]:
# Cargamos el detector de caras de dlib (shape_predictor_68_face_landmarks.dat)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Inicializamos la cámara
cap = cv2.VideoCapture(0)  # 0 representa la cámara web predeterminada, puedes cambiarlo si tienes múltiples cámaras

while True:
    # Capturamos un fotograma de la cámara
    ret, frame = cap.read()

    if not ret:
        print("No se pudo capturar un fotograma de la cámara.")
        break

    # Detección de caras en el fotograma
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # Almacenar caras detectadas
    detected_faces = []

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        detected_faces.append((x, y, w, h))

    # Si se detectaron exactamente dos caras, intercambiarlas y redimensionar al tamaño de la cara más grande
    if len(detected_faces) == 2:
        x1, y1, w1, h1 = detected_faces[0]
        x2, y2, w2, h2 = detected_faces[1]

        # Verificar que las dimensiones de las caras sean válidas
        if w1 > 0 and h1 > 0 and w2 > 0 and h2 > 0:
            # Calcular el área de las caras
            area1 = w1 * h1
            area2 = w2 * h2

            # Elegir la cara más grande y su ubicación
            if area1 > area2:
                target_x, target_y, target_w, target_h = x1, y1, w1, h1
                source_x, source_y, source_w, source_h = x2, y2, w2, h2
            else:
                target_x, target_y, target_w, target_h = x2, y2, w2, h2
                source_x, source_y, source_w, source_h = x1, y1, w1, h1

            # Verificar que las caras redimensionadas se ajusten al límite del marco
            target_x = max(0, target_x)
            target_y = max(0, target_y)
            target_x = min(frame.shape[1] - target_w, target_x)
            target_y = min(frame.shape[0] - target_h, target_y)

            source_x = max(0, source_x)
            source_y = max(0, source_y)
            source_x = min(frame.shape[1] - target_w, source_x)
            source_y = min(frame.shape[0] - target_h, source_y)

            # Redimensionar ambas caras al tamaño de la cara más grande
            face1_resized = cv2.resize(frame[source_y:source_y + source_h, source_x:source_x + source_w], 
                                       (target_w, target_h))
            face2_resized = cv2.resize(frame[target_y:target_y + target_h, target_x:target_x + target_w], 
                                       (source_w, source_h))

            # Intercambiar las caras escaladas usando la función addWeighted
            alpha = 1  
            frame[target_y:target_y + target_h, target_x:target_x + target_w] = cv2.addWeighted(
                frame[target_y:target_y + target_h, target_x:target_x + target_w], 1 - alpha, 
                face1_resized, alpha, 0
            )
            frame[source_y:source_y + source_h, source_x:source_x + source_w] = cv2.addWeighted(
                frame[source_y:source_y + source_h, source_x:source_x + source_w], 1 - alpha, 
                face2_resized, alpha, 0
            )

    # Dibujar un rectángulo alrededor de las caras
    for x, y, w, h in detected_faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Mostrar el fotograma con las caras intercambiadas y escaladas
    cv2.imshow('Intercambio de caras en tiempo real', frame)

    # Esc para finalizar
    tec = cv2.waitKey(40)
    if tec == 27:  # Esc
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()
